In [6]:
import random

import bamt.preprocessors
import bamt.networks as Nets
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, KBinsDiscretizer
from itertools import product
from pgmpy.estimators import  K2Score
import numpy as np
import seaborn as sns
import importlib
import matplotlib.pyplot as plt

from auxiliary.auxiliary import *

plt.rcParams.update({'font.size': 14})

imagespath = 'C:/Users/Acer/PycharmProjects/BAMTclone/BAMT/experiments2/images2'

In [7]:
n = 2000
np.random.seed(42)
#independent
data = pd.DataFrame(data=np.random.beta(0.95, 0.5, size=n)*6-3, columns=['origin1'])
data['origin2'] = np.random.beta(0.4,1, size=n)*2
data['origin3'] = np.random.beta(0.9,2, size=n)*4
#dependent
#1-dependent
data['cubed_o1'] = data['origin1']**3/50 + np.random.normal(scale=0.1, size=n)
data['negative_root3_o2'] = -np.sign(data['origin2'])*np.power(np.abs(data['origin2']), 1/3) + np.random.normal(scale=0.1, size=n)
data['hyperbola_o3'] = 1./(data['origin3']+1) + np.random.normal(scale=0.2, size=n)
#2-dependent
data['combined_cn'] = data['cubed_o1'] + data['negative_root3_o2']
data['combined_nh'] = data['hyperbola_o3'] + data['negative_root3_o2']
data['combined_ch'] = data['hyperbola_o3'] + data['cubed_o1']
#3-dependent
data['combined_cno3'] = data['cubed_o1'] + data['negative_root3_o2'] + data['origin3']
data['combined_co2h'] = data['cubed_o1'] + data['origin2'] + data['hyperbola_o3']
data['combined_o1nh'] = data['origin1'] + data['negative_root3_o2'] + data['hyperbola_o3']

data.head(10)

,origin1,origin2,origin3,cubed_o1,negative_root3_o2,hyperbola_o3,combined_cn,combined_nh,combined_ch,combined_cno3,combined_co2h,combined_o1nh
0,2.119492,0.737207,0.312454,0.058769,-0.992276,0.488902,-0.933507,-0.503373,0.547671,-0.621053,1.284878,1.616118
1,-2.625095,0.031537,0.776028,-0.293367,-0.484123,0.586394,-0.777490,0.102271,0.293027,-0.001462,0.324564,-2.522824
2,-2.894855,0.009888,1.209078,-0.468985,-0.142113,0.602876,-0.611097,0.460763,0.133891,0.597981,0.143779,-2.434091
3,2.688882,0.292918,0.395079,0.376103,-0.800458,0.684562,-0.424355,-0.115896,1.060665,-0.029276,1.353584,2.572987
4,1.990172,0.243591,2.290462,0.272118,-0.483364,0.209023,-0.211245,-0.274341,0.481141,2.079217,0.724732,1.715831
5,0.055618,1.007663,0.193023,-0.028964,-0.966945,0.566283,-0.995909,-0.400662,0.537319,-0.802887,1.544982,-0.345044
6,1.978326,1.391204,1.796556,0.277852,-1.020400,0.693850,-0.742548,-0.326550,0.971702,1.054008,2.362905,1.651775
7,2.810375,0.138295,1.485132,0.336994,-0.581798,0.397911,-0.244804,-0.183887,0.734905,1.240328,0.873199,2.626487
8,1.026379,0.008179,2.807084,-0.097354,-0.313829,0.172240,-0.411183,-0.141589,0.074885,2.395901,0.083065,0.884790
9,-0.542534,0.038656,1.574959,-0.017011,-0.372894,0.700862,-0.389905,0.327968,0.683852,1.185054,0.722508,-0.214566


In [17]:
qresult = construct_by_quantiles(data, None)
uresult = construct_by_uniform(data, None)
kresult = construct_by_kmeans(data, [])
kbn = kresult['bn']

#c=kbn.plot2(imagespath , "kbn16.html", custom_mapper={'marker': {0: 'Zero', 1: 'One', 2: 'Two'}})
#c=kbn.plot2("visualization_results" , "k_before1.html", custom_mapper={'marker': {0: 'Zero', 1: 'One', 2: 'Two'}})
#c=kbn.plot2("visualization_results_new" , "k_after1.html", custom_mapper={'marker': {0: 'Zero', 1: 'One', 2: 'Two'}})

In [9]:
kencoder, data_kdiscretized_enc = kresult['encoder'], kresult['disc_data']

In [10]:
#print_histogram('origin1', data, data_kdiscretized_enc, kencoder)

In [11]:
#('hyperbola', data, data_kdiscretized_enc, kencoder)

In [12]:
#print_histogram('negative_root3', data, data_kdiscretized_enc, kencoder)

In [13]:
#print_histogram('cubed', data, data_kdiscretized_enc, kencoder)

In [14]:
#c=kbn.plot_for_specific_nodes(['marker0', 'marker1', 'marker2'], imagespath , "subkbn16.html", custom_mapper={'marker': {0: 'Zero', 1: 'One', 2: 'Two'}})

In [19]:
relationships_1dep = [
    ['origin1', 'cubed_o1', 1],
    ['origin2', 'negative_root3_o2', -1],
    ['origin3', 'hyperbola_o3', -1]]

relationships_2dep = [
    ['cubed_o1', 'combined_cn', 1],
    ['negative_root3_o2', 'combined_cn', 1],
    ['origin1', 'combined_cn', 1],
    ['origin2', 'combined_cn', -1],

    ['negative_root3_o2', 'combined_nh', 1],
    ['hyperbola_o3', 'combined_nh', 1],
    ['origin2', 'combined_nh', -1],
    ['origin3', 'combined_nh', -1],

    ['cubed_o1', 'combined_ch', 1],
    ['hyperbola_o3', 'combined_ch', 1],
    ['origin1', 'combined_ch', 1],
    ['origin3', 'combined_ch', -1],
]

relationships_3dep = [
    ['cubed_o1', 'combined_cno3', 1],
    ['negative_root3_o2', 'combined_cno3', 1],
    ['origin1', 'combined_cno3', 1],
    ['origin2', 'combined_cno3', -1],
    ['origin3', 'combined_cno3', 1],

    ['cubed_o1', 'combined_co2h', 1],
    ['hyperbola_o3', 'combined_co2h', 1],
    ['origin1', 'combined_co2h', 1],
    ['origin2', 'combined_co2h', 1],
    ['origin3', 'combined_co2h', -1],

    ['negative_root3_o2', 'combined_o1nh', 1],
    ['hyperbola_o3', 'combined_o1nh', 1],
    ['origin1', 'combined_o1nh', 1],
    ['origin2', 'combined_o1nh', -1],
    ['origin3', 'combined_o1nh', -1],
]
realtionships = relationships_1dep + relationships_2dep + relationships_3dep

true_edges = list()
true_edges_bydep = [None, None, None]
states=[0,1,2]
def get_true_edges(relationships):
    result = list()
    for rel in relationships:
        result+=([[rel[0]+str(i), rel[1]+str(j)] for i, j in zip(states, states[::rel[2]])])
    return result

for i, rels in enumerate([relationships_1dep, relationships_2dep, relationships_3dep]):
    true_edges_bydep[i] = get_true_edges(rels)
true_edges = sum(true_edges_bydep, start=[])

In [20]:
true_edges

[['origin10', 'cubed_o10'],
 ['origin11', 'cubed_o11'],
 ['origin12', 'cubed_o12'],
 ['origin20', 'negative_root3_o22'],
 ['origin21', 'negative_root3_o21'],
 ['origin22', 'negative_root3_o20'],
 ['origin30', 'hyperbola_o32'],
 ['origin31', 'hyperbola_o31'],
 ['origin32', 'hyperbola_o30'],
 ['cubed_o10', 'combined_cn0'],
 ['cubed_o11', 'combined_cn1'],
 ['cubed_o12', 'combined_cn2'],
 ['negative_root3_o20', 'combined_cn0'],
 ['negative_root3_o21', 'combined_cn1'],
 ['negative_root3_o22', 'combined_cn2'],
 ['origin10', 'combined_cn0'],
 ['origin11', 'combined_cn1'],
 ['origin12', 'combined_cn2'],
 ['origin20', 'combined_cn2'],
 ['origin21', 'combined_cn1'],
 ['origin22', 'combined_cn0'],
 ['negative_root3_o20', 'combined_nh0'],
 ['negative_root3_o21', 'combined_nh1'],
 ['negative_root3_o22', 'combined_nh2'],
 ['hyperbola_o30', 'combined_nh0'],
 ['hyperbola_o31', 'combined_nh1'],
 ['hyperbola_o32', 'combined_nh2'],
 ['origin20', 'combined_nh2'],
 ['origin21', 'combined_nh1'],
 ['origin22

In [22]:
for bnet, name in zip([uresult['bn'], qresult['bn'], kresult['bn']], ['Uniform', 'Quantlile', 'Kmeans']):
    for i in range(3):
        res = calculate_ratio(bnet.edges, true_edges_bydep[i]), calculate_reversed_ratio(bnet.edges, true_edges_bydep[i])
        print(f' Strategy {name} gives BN in which {i+1}-dep features` true relationships are found with rates {res}')
    res = calculate_ratio(bnet.edges, true_edges), calculate_reversed_ratio(bnet.edges, true_edges)
    print(f' Strategy {name} gives BN in which true relationships are found with rates {res}')


#calculate_ratio(uresult['bn'].edges, true_edges), calculate_reversed_ratio(uresult['bn'].edges, true_edges)

 Strategy Uniform gives BN in which 1-dep features` true relationships are found with rates (0.5555555555555556, 0.2222222222222222)
 Strategy Uniform gives BN in which 2-dep features` true relationships are found with rates (0.5833333333333334, 0.05555555555555555)
 Strategy Uniform gives BN in which 3-dep features` true relationships are found with rates (0.15555555555555556, 0.1111111111111111)
 Strategy Uniform gives BN in which true relationships are found with rates (0.36666666666666664, 0.1)
 Strategy Quantlile gives BN in which 1-dep features` true relationships are found with rates (0.6666666666666666, 0.2222222222222222)
 Strategy Quantlile gives BN in which 2-dep features` true relationships are found with rates (0.3888888888888889, 0.19444444444444445)
 Strategy Quantlile gives BN in which 3-dep features` true relationships are found with rates (0.15555555555555556, 0.044444444444444446)
 Strategy Quantlile gives BN in which true relationships are found with rates (0.3, 0.1

In [ ]:
calculate_ratio(qresult['bn'].edges, true_edges), calculate_reversed_ratio(qresult['bn'].edges, true_edges)

In [ ]:
calculate_ratio(kbn.edges, true_edges), calculate_reversed_ratio(kbn.edges, true_edges)

Эксперименты с зашумлением